# Constraint Satisfaction Problem: Personalized Stock Portfolio

The rise of retail investors in financial markets has highlighted a need for personalized portfolio guidance. This paper presents a novel algorithm that uses refined approach to the Constraint Satisfaction Problem (CSP) framework to optimize stock portfolio allocations based on individual preferences, such as risk tolerance, return expectations, and sustainability ratings. By modeling stocks as variables, allocations as domains, and financial goals as constraints, we ensure portfolios align with investor objectives. Our approach bridges a gap in CSP literature by addressing portfolio optimization as a resource allocation problem, offering a practical tool for retail investors to make informed decisions and enhance financial inclusion.  

**Disclaimer: The content presented in this notebook is for educational and illustrative purposes only. It does not constitute financial advice or a recommendation to buy, sell, or hold any securities or investments.  Investments in financial markets involve risks, including the potential loss of capital.**  

**Note**: In order to run this notebook, you will need to upload the associated file with stock data `ST449_Project_DATA.xlsx` to the working directory in Google Colab.
_______________________________

## Install libraries & packages
  * `cvxpy`: Solves convex optimization problems. We use this over the `python-constraint` AIMA package because it allows for continuous domains and constraints represented by linear inequalities [(Source)](https://github.com/cvxpy/cvxpy).
  * `pandas`: An open source data analysis and manipulation tool [(Source)](https://pandas.pydata.org/).
  * `numpy`: Python library used for working with arrays. It also has functions for working in domain of linear algebra, fourier transform, and matrices [(Source)](https://numpy.org/).

In [ ]:
!pip install cvxpy
!pip install pandas
!pip install numpy

## Import packages and data
These cells import relevant packages and load the data:
- Reads the stock data into a Pandas DataFrame.
- Displays the first few rows of the dataset to verify successful loading.
- This step ensures the data includes relevant columns such as returns, volatility, and ESG scores.

In [ ]:
import cvxpy as cp
import pandas as pd
import numpy as np

In [ ]:
print("#" * 30 + " IMPORT DATA " + "#" * 30)
data = pd.read_excel('GroupB.xlsx')
print(data.head())
print("\n" + "#" * 70 + "\n")

############################## IMPORT DATA ##############################
           Name    1yret    5yret   10yret  Volatility    Beta   ESG
0  1AMZLNEquity  44.9844  44.9844  44.9844    0.951782  0.8733  5.35
1  1COILNEquity  39.6407  39.6407  39.6407    5.110986  1.8795  5.78
2  1MCSLNEquity  -2.7363  -3.0468  -7.3623    1.307267 -0.2019  4.38
3  1PASLNEquity  12.1694  -4.1893 -14.0263    3.630729 -0.6350  4.32
4   1SNLNEquity  23.8405  23.8405  23.8405    4.223100  0.4398  5.28

######################################################################



## Create the PortfolioOptimizer class

In [ ]:
#Define the class and the __init__ method
class PortfolioOptimizer:
    def __init__(self, data, preferences=None):
        """
        Initialize the PortfolioOptimizer class.
        Args:
            data (DataFrame): The input dataset containing stock information.
            preferences (dict, optional): User-specified preferences. If None, preferences will be queried.
        """
        self.df = data.reset_index(drop=True)
        self.constraints = []
        self.w = None
        self.solution_weights = None
        self.portfolio_metrics = {}

        # Set preferences
        if preferences is not None:
            self.preferences = preferences
        else:
            self.preferences = {}

## Prompt the user for their preferences
Below is an interactive prompt that queries the user on their preferences.
  * **Investment horizon:** Asks user what their investment horizon is: 1, 5, 10 years
  * **Volatility and return**: Asks user what their desired maximum volatility. Based on this risk preference, the program automatically selects a desired return. This is to prevent selection of infeasible constrains, like very low volatility but a very high desired return.
  * **Sustainability rating**: Asks user what their preference is on sustainability rating of their portfolio out of 10 and assigns a corresponding ESG minimum based on their preference.
    * **Options**: 1 - Low (1-3),  2 - Medium (4-6), 3 - High - (7-10). The max score assignment is 6, as the portfolio fails with a higher ESG constraint.
    * ESG scores "*...are designed to transparently and objectively measure a company's relative ESG performance, commitment and effectiveness across 10 main themes (emissions, environmental product innovation, human rights, shareholders, etc.) based on publicly-reported data.*" [(Source)](https://www.lseg.com/en/data-analytics/sustainable-finance/esg-scores#methodology).

* **Beta**: Asks user how closely they would like the portfolio to follow the market, which sets the Beta value.
    * **Options**: 1 - Defensive (Beta ≈ 0.8), 2 - Neutral (Beta ≈ 1), 3 - Aggressive (Beta ≈ 1.2).
    * Beta is "*...a measurement of its volatility of returns relative to the entire market.*" [(Source)](https://corporatefinanceinstitute.com/resources/valuation/what-is-beta-guide/).

In [ ]:
# Extend the PortfolioOptimizer class
class PortfolioOptimizer(PortfolioOptimizer):
    """Extend the class to query preferences from an individual
    about their portfolio preferences."""

    def query_preferences(self):
        self.preferences = {}  # Initialize preferences dictionary

        # Returns timeline
        print("How long are you planning to hold this portfolio?")
        print("1: Short-Term (1 year return)")
        print("2: Medium-Term (5 year return)")
        print("3: Long-Term (10 year return)")
        timeline_choice = int(input("Enter your choice (1/2/3): "))
        if timeline_choice == 1:
            self.preferences['time_horizon'] = '1yret'
        elif timeline_choice == 2:
            self.preferences['time_horizon'] = '5yret'
        else:
            self.preferences['time_horizon'] = '10yret'

        # Risk tolerance
        print("How much risk are you comfortable taking?")
        print("1: Low (1% volatility, >6% return) - Recommended for long term portfolios")
        print("2: Medium (5% volatility, >10% return) - Recommended for medium term portfolios")
        print("3: High (10%% volatility, >15% return) - Recommended for short term portfolios")
        risk_choice = int(input("Enter your choice (1/2/3): "))
        if risk_choice == 1:
            self.preferences['volatility'] = 1
            self.preferences['total_return'] = 6
        elif risk_choice == 2:
            self.preferences['volatility'] = 5
            self.preferences['total_return'] = 10
        else:
            self.preferences['volatility'] = 10
            self.preferences['total_return'] = 15

        # Beta
        print("How closely do you want your portfolio to follow the market?")
        print("1: Defensive (Beta < 0.8)")
        print("2: Neutral (Beta ≈ 1)")
        print("3: Aggressive (Beta > 1.2)")
        beta_choice = int(input("Enter your choice (1/2/3): "))
        if beta_choice == 1:
            self.preferences['beta'] = 0.8
        elif beta_choice == 2:
            self.preferences['beta'] = 1
        else:
            self.preferences['beta'] = 1.2

        # ESG rating
        print("How important is sustainability (ESG rating) in your portfolio, on a scale of 1-10?")
        print("1: Low (1–3)")
        print("2: Medium (4–6)")
        print("3: High (7–10)")
        esg_choice = int(input("Enter your choice (1/2/3): "))
        if esg_choice == 1:
            self.preferences['sustainability'] = 1
        elif esg_choice == 2:
            self.preferences['sustainability'] = 3
        else:
            self.preferences['sustainability'] = 6

        print("\nYour selected preferences:")
        print(self.preferences)

## Set up and defining constraints
This cell prepares the data and sets up the decision variable for the optimization problem, then defines the constraints based on preferences.

### 1. Setting up the weight data frame

1. **Extract Key Columns:**
   - **`time_horizon_col`:** Extracts the column name based on the user's selected time horizon (e.g., `'10yret'` for 10-year returns).
   - **`returns`:** Converts the selected return column into an array to use in calculations.
   - **`volatilities`:** Extracts the volatility column as an array.
   - **`esgs`:** Extracts the ESG column as an array.
   - All three arrays have the same shape, corresponding to the number of stocks included.

2. **Determine Number of Stocks:**
   - `n = len(df)` computes the number of stocks in the subset, which will be used to define the dimensions of the optimization variable.

3. **Define Decision Variable:**
   - `w = cp.Variable(n, nonneg=True)` creates a CVXPY variable for the portfolio weights:
     - `n` is the number of stocks.
     - `nonneg=True` ensures all weights are non-negative.
     - Each element of `w` represents the proportion of the portfolio allocated to a specific stock.

<br />

### 2. Define constraints given the user's preferences

Next, we define the constraints for the portfolio optimization problem. Each constraint ensures that the portfolio satisfies specific investor preferences or rules.

1. **Initialize Constraints:**
   - An empty list `constraints` is created to store all the constraints for the optimization problem.

2. **Sum of Weights = 1:**
   - `constraints.append(cp.sum(w) == 1)` ensures that the total portfolio allocation equals 100% (i.e., the sum of all weights is 1).

3. **Maximum Allocation of Any Single Stock = 10%:**
   - `constraints.append(w <= 0.10)` restricts the allocation of any single stock to a maximum of 10% of the total portfolio. This ensures the portfolio is diverse and made up of multiple stocks, rather than one stock being assigned a weight of 1.

4. **Minimum Return:**
   - `constraints.append(returns @ w >= preferences['total_return'])` ensures that the weighted portfolio return meets or exceeds the required total return specified in the preferences (`preferences['total_return']`).

5. **Maximum Volatility:**
   - `constraints.append(volatilities @ w <= preferences['volatility'])` ensures that the weighted portfolio volatility remains below the maximum volatility threshold specified in the preferences (`preferences['volatility']`).

6. **Minimum ESG:**
   - `constraints.append(esgs @ w >= preferences['sustainability'])` ensures that the weighted ESG score of the portfolio meets or exceeds the minimum sustainability score specified in the preferences (`preferences['sustainability']`).

### Summary:
- These constraints collectively ensure that the portfolio satisfies diversification (via weights), return, risk, and sustainability requirements.
- By appending them to the `constraints` list, they are ready to be passed to the optimization solver.


In [ ]:
class PortfolioOptimizer(PortfolioOptimizer):

  def extract_data(self):
        """Extracts data from data based on the user's preferred
        method of calculating returns."""

        # extract numeric data
        time_horizon_col = self.preferences['time_horizon']
        self.returns = self.df[time_horizon_col].values
        self.volatilities = self.df['Volatility'].values
        self.esgs = self.df['ESG'].values
        self.betas = self.df['Beta'].values

        # number of stocks we are including
        self.n = len(self.df)

        # Define a CVXPY variable for each stock's allocation
        self.w = cp.Variable(self.n, nonneg=True)

  def setup_constraints(self):
      """Implements constraints based on previous preferences."""
      # set-up beta tolerance
      beta_tolerance = 0.05

      # constraints
      self.constraints = [
                  cp.sum(self.w) == 1,  # Constraint 1: Sum of weights = 1
                  self.w <= 0.10,  # Constraint 2: Max allocation of any stock = 10%
                  self.returns @ self.w >= self.preferences['total_return'],  # 3: Satisfies min. return
                  self.volatilities @ self.w <= self.preferences['volatility'],  #4:  Satisfies max. volatility
                  self.esgs @ self.w >= self.preferences['sustainability'],  # 5: Satisfies min. ESG
                  self.betas @ self.w >= self.preferences['beta'] - beta_tolerance,  # 6: Beta range
                  self.betas @ self.w <= self.preferences['beta'] + beta_tolerance
              ]

## Solve the optimization problem and output results
  * **`cp.Maximize(returns @ w)`** - This is the objective function used in the problem. In this case, we would like to maximize the weighted returns.
  * **`cp.Problem(obj, constraints)`** - Given our objective function and constraints, defined the problem.
  * **`prob.solve(solver=cp.GLPK)`** - Solve the problem using the `cp.GLPK` solver.
    * GLPK stands for GNU Linear Programming Kit. It is a solver that is well-suited for solving linear programming problems, where both the objective function (maximizing portfolio returns) and constraints (e.g., weights summing to 1) are linear.
    * We chose this because support for continuous variables makes it ideal for this problem’s continuous domain.

Then outputs results including total portfolio metrics and individual stock allocations and details.

In [ ]:
class PortfolioOptimizer(PortfolioOptimizer):

    # optimize the portfolio
    def optimize_portfolio(self):
        """Optimize the portfolio based on preferences and constraints."""
        # ensure data and constraints are set up
        self.extract_data()
        self.setup_constraints()

        # define the objective function
        objective = cp.Maximize(self.returns @ self.w)

        # solve the optimization problem
        problem = cp.Problem(objective, self.constraints)
        problem.solve(solver=cp.GLPK)

        # store the solution
        self.solution_weights = self.w.value
        self.portfolio_metrics = {
            'volatility': self.volatilities @ self.solution_weights,
            'esg': self.esgs @ self.solution_weights,
            'beta': self.betas @ self.solution_weights
        }
        print("Solver Status:", problem.status)
        print("Optimal Value:", problem.value)

    # return the portfolio
    def get_optimal_portfolio(self):
      """Retrieve the list of stocks with weights > 0 and display portfolio metrics."""

      # Compute portfolio metrics
      portfolio_volatility = self.volatilities @ self.solution_weights
      portfolio_esg = self.esgs @ self.solution_weights
      portfolio_beta = self.betas @ self.solution_weights
      portfolio_return = self.returns @ self.solution_weights

      # Print portfolio metrics
      print()
      print(f"Portfolio Return: {portfolio_return:.2f} (Min Return: {self.preferences['total_return']})")
      print(f"Portfolio Volatility: {portfolio_volatility:.2f} (Max Allowed: {self.preferences['volatility']})")
      print(f"Portfolio ESG: {portfolio_esg:.2f} (Min Required: {self.preferences['sustainability']})")
      print(f"Portfolio Beta: {portfolio_beta:.2f} (Range ± 0.05: {self.preferences['beta']})")
      print("\nOptimal Portfolio Allocation:\n")

      # Print stock details
      weights_with_details = [
        (
            self.df['Name'][i],
            self.solution_weights[i],
            self.df[self.preferences['time_horizon']].iloc[i],
            self.df['ESG'].iloc[i],
            self.df['Volatility'].iloc[i],
            self.df['Beta'].iloc[i]
        )
        for i in range(self.n)
        if self.solution_weights[i] > 1e-12  # Small threshold to avoid numerical artifacts
    ]

    # Print header
      print(f"{'Name':<20} {'Weight':<10} {'Return':<10} {'ESG':<10} {'Volatility':<10} {'Beta':<10}")
      print("-" * 70)

      # Print table rows
      for name, weight, stock_return, esg, volatility, beta in sorted(weights_with_details, key=lambda x: x[1], reverse=True):
          print(f"{name:<20} {weight:<10.6f} {stock_return:<10.2f} {esg:<10.2f} {volatility:<10.2f} {beta:<10.2f}")

## Create a final function to return portfolio allocations
 Returns ideal stock allocation based on preferences as well as the portfolio overview.

 Preferences are initially set to 'none', allowing the user to be queried for preferences; with query responses, it is unlikely the code will generate an infeasible solution. However, if the user has more specific preferences (e.g., a volatility between 1% and 5% or a higher return preference), they are able to input it as a dictionary; they simply run the risk of creating an impossible set of constraints.

In [ ]:
# Put it all together into a function.

def get_portfolio(data, preferences=None):
  """Arguments: data and, if known, preferences. Note that inputting preferences
  directly may result in an infeasible portfolio due to conflicting constraints.
  If unknown, leave preferences empty, and answer querying preferences.

  Returns ideal stock allocation based on preferences as well as the portfolio
  overview."""
  optimizer = PortfolioOptimizer(data, preferences=preferences)
  if optimizer.preferences == {}:
    optimizer.query_preferences()
  optimizer.optimize_portfolio()
  portfolio = optimizer.get_optimal_portfolio()
  return portfolio

## Simulated scenarios
__________

**Investor One (Included in Paper):**

This investor wants a long term investment that is minimal risk. They care a lot about the environment, and want a porfolio that is not too closely tied to the market to avoid the market's dips and falls. Therefore, in answering the query questions, they answer: long-term, low risk, defensive, and high ESG. This results in a preference dictionary that looks as follows: {'time_horizon': '10yret', 'volatility': 1, 'total_return': 6, 'beta': 0.8, 'sustainability': 6}

In [ ]:
get_portfolio(data, {'time_horizon': '10yret', 'volatility': 1, 'total_return': 6, 'beta': 0.8, 'sustainability': 6})

Solver Status: optimal
Optimal Value: 35.26970276540037

Portfolio Return: 35.27 (Min Return: 6)
Portfolio Volatility: 1.00 (Max Allowed: 1)
Portfolio ESG: 6.00 (Min Required: 6)
Portfolio Beta: 0.75 (Range ± 0.05: 0.8)

Optimal Portfolio Allocation:

Name                 Weight     Return     ESG        Volatility Beta      
----------------------------------------------------------------------
GABILNEquity         0.100000   70.54      5.55       0.86       0.41      
GOO2LNEquity         0.100000   61.71      6.34       1.98       0.80      
1AMZLNEquity         0.100000   44.98      5.35       0.95       0.87      
AOFLNEquity          0.100000   24.40      6.95       0.57       0.45      
DECLNEquity          0.100000   37.62      4.67       1.09       1.30      
IGLNLNEquity         0.100000   7.88       6.81       0.24       0.52      
SPALLNEquity         0.100000   0.88       6.20       0.65       1.26      
EJFILNEquity         0.100000   31.07      5.43       0.34       0.35

**Investor Two:**

This investor is risky and wants a high yield in a short amount of time. They want a portfolio that aggressively follows the market trends and does not care about ESG ratings. In answering the query questions, they answer: short-term, high risk, aggressive, and low ESG. This results in a preference dictionary that looks as follows: {'time_horizon': '1yret', 'volatility': 10, 'total_return': 15, 'beta': 1.2, 'sustainability': 2}


In [ ]:
get_portfolio(data, {'time_horizon': '1yret', 'volatility': 10, 'total_return': 15, 'beta': 1.2, 'sustainability': 2})

Solver Status: optimal
Optimal Value: 89.86485707811185

Portfolio Return: 89.86 (Min Return: 15)
Portfolio Volatility: 4.85 (Max Allowed: 10)
Portfolio ESG: 4.51 (Min Required: 2)
Portfolio Beta: 1.15 (Range ± 0.05: 1.2)

Optimal Portfolio Allocation:

Name                 Weight     Return     ESG        Volatility Beta      
----------------------------------------------------------------------
URULNEquity          0.100000   99.62      5.89       9.26       0.95      
JUSTLNEquity         0.100000   92.39      3.83       1.35       1.09      
OXBLNEquity          0.100000   90.57      2.99       5.59       0.95      
CURYLNEquity         0.100000   87.60      5.84       0.83       1.01      
GNCLNEquity          0.100000   100.55     5.35       2.14       0.76      
TSL2LNEquity         0.100000   71.82      4.91       9.64       1.81      
NEXNLNEquity         0.100000   95.72      1.57       5.10       0.82      
AQXLNEquity          0.100000   90.76      5.30       3.81       1.

**Investor three:**

This investor wants a return within 5 years, and is therefore willing to accept a medium risk and return.

They want their portfolio to roughly follow the market, and they care only somewhat about ESG ratings. In answering the query questions, they answer: medium term, medium risk, neutral on the market, and medium ESG. This results in a preference dictionary that looks as follows: {'time_horizon': '5yret', 'volatility': 5, 'total_return': 10, 'beta': 1, 'sustainability': 4}

In [ ]:
get_portfolio(data, {'time_horizon': '5yret', 'volatility': 5, 'total_return': 10, 'beta': 1, 'sustainability': 4})

Solver Status: optimal
Optimal Value: 73.97813880460855

Portfolio Return: 73.98 (Min Return: 10)
Portfolio Volatility: 5.00 (Max Allowed: 5)
Portfolio ESG: 5.74 (Min Required: 4)
Portfolio Beta: 0.95 (Range ± 0.05: 1)

Optimal Portfolio Allocation:

Name                 Weight     Return     ESG        Volatility Beta      
----------------------------------------------------------------------
1TSLLNEquity         0.100000   65.22      6.39       2.97       1.06      
3LALLNEquity         0.100000   76.02      5.36       5.32       0.99      
GOO3LNEquity         0.100000   77.18      6.23       5.36       1.03      
TSL2LNEquity         0.100000   71.82      4.91       9.64       1.81      
ADVTLNEquity         0.100000   91.73      5.31       2.89       0.57      
AOFLNEquity          0.100000   67.21      6.95       0.57       0.45      
AAP3LNEquity         0.100000   66.96      5.82       4.33       1.03      
4BBLNEquity          0.100000   77.66      4.56       4.62       0.51 

_____________
## Try it yourself!

Simply input `get_portfolio(data)`, answer the questions, and find your ideal portfolio diversification for stocks in the London Stock Exchange!

In [ ]:
get_portfolio(data)

How long are you planning to hold this portfolio?
1: Short-Term (1 year return)
2: Medium-Term (5 year return)
3: Long-Term (10 year return)
Enter your choice (1/2/3): 1
How much risk are you comfortable taking?
1: Low (1% volatility, >6% return) - Recommended for long term portfolios
2: Medium (5% volatility, >10% return) - Recommended for medium term portfolios
3: High (10%% volatility, >15% return) - Recommended for short term portfolios
Enter your choice (1/2/3): 1
How closely do you want your portfolio to follow the market?
1: Defensive (Beta < 0.8)
2: Neutral (Beta ≈ 1)
3: Aggressive (Beta > 1.2)
Enter your choice (1/2/3): 3
How important is sustainability (ESG rating) in your portfolio, on a scale of 1-10?
1: Low (1–3)
2: Medium (4–6)
3: High (7–10)
Enter your choice (1/2/3): 2

Your selected preferences:
{'time_horizon': '1yret', 'volatility': 1, 'total_return': 6, 'beta': 1.2, 'sustainability': 3}
Solver Status: optimal
Optimal Value: 57.40196702843824

Portfolio Return: 57.40